# Riot APi로 유저정보 받기

In [1]:
import requests
from urllib import parse
import time
import json
import pandas as pd
from urllib.request import urlopen

In [2]:
key_fd = open('../keys/api_key.txt', mode='r')
api_key = key_fd.read(100)
key_fd.close()

In [3]:
request_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key  #Riot API 키   
}

In [4]:
champion_df = pd.read_csv('../00. data/champions.csv', index_col = 0)
champion_df.head()

,name,key,tags
0,아트록스,266,"Fighter, Tank"
1,아리,103,"Mage, Assassin"
2,아칼리,84,Assassin
3,알리스타,12,"Tank, Support"
4,아무무,32,"Tank, Mage"


In [5]:
item_df = pd.read_csv('../00. data/items.csv', index_col = 0)
item_df.head()

,name,key,gold,tags
0,장화,1001,300,Boots
1,요정의 부적,1004,250,ManaRegen
2,원기 회복의 구슬,1006,150,HealthRegen
3,거인의 허리띠,1011,900,Health
4,민첩성의 망토,1018,600,CriticalStrike


In [6]:
# 유저의 암호화된 아이디를 불러오는 함수 유저의 암호화된 아이디를 불러오는 함수
def searchUserId(nickname):
    user_data = requests.get("https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + parse.quote(nickname), headers=request_headers).json()
    userId = user_data['id']
    userAccountId = user_data['accountId']
    userPuuid = user_data['puuid'] 
    return userId, userAccountId, userPuuid
# 한유저의 특정 챔피언 매치ID들을 불러오는 함수
def searchChampMatchId(accountId, champion):
    championKey = champion_df['key'][champion_df['key'][champion_df['name'] == champion].index[0]] # 챔피언 키구하기
    match_data = requests.get("https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/" + accountId + "?champion=" + str(championKey),headers=request_headers).json()['matches']
    match_list = []
    for i in range(len(match_data)):
        match = match_data[i]['gameId']
        match_list.append(match)
    return match_list
# 한유저의 매치 ID들을 불러오는 함수
def searchMatchId(accountId):
    match_data = requests.get("https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/" + accountId,headers=request_headers).json()['matches']
    match_list = []
    for i in range(len(match_data)):
        match = match_data[i]['gameId']
        match_list.append(match)
    return match_list
# 챔피언키로 챔피언 이름 불러오는 함수
def searchChampion(championKey):
    champ = champion_df['name'][champion_df['name'][champion_df['key'] == championKey].index[0]]
    return champ  
# 아이템키로 아이템 이름 불러오는 함수
def searchItem(itemKey):
    item = item_df['name'][item_df['name'][item_df['key'] == itemKey].index[0]]
    return item  

In [7]:
userId, userAccountId, userPuuid = searchUserId('Giyeom') # 유저의 닉네임 Giyeom
match_list = searchChampMatchId(userAccountId, '나르') # 유저의 챔피언 나르로 설정
len(match_list) # 최대가 100개

100

In [8]:
match_list2 = searchMatchId(userAccountId)

In [9]:
match_list[0]

5291438564

In [10]:
# sample_a
a = requests.get('https://kr.api.riotgames.com/lol/match/v4/matches/' + str(match_list[0]), headers=request_headers).json()

In [11]:
list(a)

['gameId',
 'platformId',
 'gameCreation',
 'gameDuration',
 'queueId',
 'mapId',
 'seasonId',
 'gameVersion',
 'gameMode',
 'gameType',
 'teams',
 'participants',
 'participantIdentities']

In [12]:
a['teams']

[{'teamId': 100,
  'win': 'Win',
  'firstBlood': False,
  'firstTower': True,
  'firstInhibitor': True,
  'firstBaron': True,
  'firstDragon': True,
  'firstRiftHerald': True,
  'towerKills': 9,
  'inhibitorKills': 2,
  'baronKills': 1,
  'dragonKills': 2,
  'vilemawKills': 0,
  'riftHeraldKills': 2,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 526, 'pickTurn': 1},
   {'championId': 64, 'pickTurn': 2},
   {'championId': 517, 'pickTurn': 3},
   {'championId': 120, 'pickTurn': 4},
   {'championId': 12, 'pickTurn': 5}]},
 {'teamId': 200,
  'win': 'Fail',
  'firstBlood': True,
  'firstTower': False,
  'firstInhibitor': False,
  'firstBaron': False,
  'firstDragon': False,
  'firstRiftHerald': False,
  'towerKills': 3,
  'inhibitorKills': 0,
  'baronKills': 0,
  'dragonKills': 1,
  'vilemawKills': 0,
  'riftHeraldKills': 0,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 360, 'pickTurn': 6},
   {'championId': 56, 'pickTurn': 7},
   {'championId': 81, 'pickTurn': 8},
   {'ch

In [13]:
a['participants'][0]

{'participantId': 1,
 'teamId': 100,
 'championId': 82,
 'spell1Id': 11,
 'spell2Id': 4,
 'stats': {'participantId': 1,
  'win': True,
  'item0': 4633,
  'item1': 4637,
  'item2': 2420,
  'item3': 3111,
  'item4': 3108,
  'item5': 0,
  'item6': 3364,
  'kills': 6,
  'deaths': 2,
  'assists': 6,
  'largestKillingSpree': 4,
  'largestMultiKill': 1,
  'killingSprees': 2,
  'longestTimeSpentLiving': 556,
  'doubleKills': 0,
  'tripleKills': 0,
  'quadraKills': 0,
  'pentaKills': 0,
  'unrealKills': 0,
  'totalDamageDealt': 175204,
  'magicDamageDealt': 141033,
  'physicalDamageDealt': 21541,
  'trueDamageDealt': 12631,
  'largestCriticalStrike': 0,
  'totalDamageDealtToChampions': 11435,
  'magicDamageDealtToChampions': 8678,
  'physicalDamageDealtToChampions': 1081,
  'trueDamageDealtToChampions': 1676,
  'totalHeal': 10449,
  'totalUnitsHealed': 1,
  'damageSelfMitigated': 16925,
  'damageDealtToObjectives': 41836,
  'damageDealtToTurrets': 5523,
  'visionScore': 11,
  'timeCCingOthers':

In [14]:
a['participantIdentities'][0]

{'participantId': 1,
 'player': {'platformId': 'KR',
  'accountId': 'Iv7fd-IJoCf76_psTEiLMYpmgNT2b-M6PwOtszP93mG_',
  'summonerName': 'KUBO SAYANO',
  'summonerId': 'iwaS4NM218w8pmuZMhJX23IaHDQPq3YO4Cgsfv9ze1GRpQ',
  'currentPlatformId': 'KR',
  'currentAccountId': 'Iv7fd-IJoCf76_psTEiLMYpmgNT2b-M6PwOtszP93mG_',
  'matchHistoryUri': '/v1/stats/player_history/KR/1794063',
  'profileIcon': 4379}}

In [15]:
# sample_a의 유저 닉네임 구하기 
users_info = a['participantIdentities']
user_list = []
for user_info in users_info:
    user_name = user_info['player']['summonerName']
    user_list.append(user_name)

In [16]:
user_list

['KUBO SAYANO',
 '도전전문유튜버',
 '킹매이커',
 '탱 푸',
 '깽판 전문가',
 'Giyeom',
 '다이아개돼지련들',
 '미아치면오픈',
 'starfired',
 'qweljkfsdv']

In [17]:
a['participants'][0]['timeline']['lane']

'JUNGLE'

In [18]:
def match_record(matchId):
    match_record = requests.get('https://kr.api.riotgames.com/lol/match/v4/matches/' + str(matchId), headers=request_headers).json()
    users_record = match_record['participants']
    users_info = match_record['participantIdentities']
    user_list = []
    for user_info in users_info:
        user_name = user_info['player']['summonerName']
        user_list.append(user_name)
    users_champion = []
    users_kda = []
    users_items = []
    users_lane = []
    for user_record in users_record:
        user_champion = searchChampion(user_record['championId'])
        user_kda = str(user_record['stats']['kills']) + '/' + str(user_record['stats']['deaths']) + '/' + str(user_record['stats']['assists'])
        user_items = []
        for i in range(6):
            itemKey = user_record['stats']['item'+str(i)]
            if itemKey == 0:
                pass
            else:
                user_items.append(searchItem(user_record['stats']['item'+str(i)]))
        user_items = ', '.join(user_items)
        user_lane = user_record['timeline']['lane']
        users_champion.append(user_champion)
        users_kda.append(user_kda)
        users_items.append(user_items)
        users_lane.append(user_lane)
    df = pd.DataFrame({
    'Nickname' : user_list,
    'Champion' : users_champion,
    'Lane' : users_lane,
    'KDA' : users_kda,
    'Items' : users_items
    })
    return df

In [19]:
# 전적 test 1
match_record(match_list[25])

,Nickname,Champion,Lane,KDA,Items
0,Yogg Saron Box,우디르,JUNGLE,6/3/5,"터보 화공 탱크, 제어 와드, 망자의 갑옷, 파수꾼의 갑옷, 루비 수정, 명석함의 ..."
1,삐용삐용이멀젼씨,루시안,MIDDLE,5/4/3,"돌풍, 광전사의 군화, 정수 약탈자, 롱소드, 도란의 검, 롱소드"
2,믿 거 빵,벨코즈,BOTTOM,4/4/9,"만년서리, 얼음 정수의 파편, 증폭의 고서, 장화"
3,황족 MID LINER,스웨인,BOTTOM,6/0/7,"충전형 물약, 만년서리, 존야의 모래시계, 마법사의 신발, 도란의 반지"
4,Giyeom,나르,TOP,4/2/6,"발걸음 분쇄기, 판금 장화, 도란의 방패, 점화석, 파수꾼의 갑옷"
5,Manila,베인,TOP,2/6/0,"충전형 물약, 도란의 검, 절정의 화살, 광전사의 군화, 몰락한 왕의 검"
6,04년생 치타,렉사이,JUNGLE,2/7/3,여신의 눈물
7,강 철 산i,트리스타나,BOTTOM,2/1/1,"도란의 검, 충전형 물약, 크라켄 학살자, 광전사의 군화, 유령 무희"
8,영탈자,판테온,BOTTOM,6/7/0,"룬 강철 어깨 갑옷, 월식, 기동력의 장화, 제어 와드, 톱날 단검, 롱소드"
9,삐리링버스기사,요네,MIDDLE,0/4/3,"도란의 방패, 불멸의 철갑궁, 광전사의 군화"


In [20]:
# 전적 test 2
match_record(match_list2[5])

,Nickname,Champion,Lane,KDA,Items
0,GreendayGreenday,피즈,MIDDLE,8/7/7,"루덴의 폭풍, 망가진 초시계, 존야의 모래시계, 명석함의 아이오니아 장화, 리치베인..."
1,핑으로대화바람,렝가,JUNGLE,16/7/6,"드락사르의 황혼검, 정수 약탈자, 징수의 총, 밤의 끝자락, 기동력의 장화, 독사의..."
2,국산망둥어,아펠리오스,BOTTOM,9/3/14,"루난의 허리케인, 무한의 대검, 크라켄 학살자, 광전사의 군화, 피바라기, B.F. 대검"
3,ganttz,쓰레쉬,BOTTOM,0/7/18,"화이트록의 갑옷, 강철의 솔라리 펜던트, 제어 와드, 덤불 조끼, 망자의 갑옷, 기..."
4,자유생성,볼리베어,TOP,6/6/5,"존야의 모래시계, 초시계, 신성한 파괴자, 파수꾼의 갑옷, 명석함의 아이오니아 장화..."
5,헌하짱짱,세트,BOTTOM,2/8/16,"터보 화공 탱크, 판금 장화, 제어 와드, 화이트록의 갑옷, 망자의 갑옷"
6,Animal Beaver,피들스틱,JUNGLE,5/8/14,"존야의 모래시계, 리안드리의 고뇌, 마법사의 신발, 제어 와드, 쓸데없이 큰 지팡이..."
7,Giyeom,레넥톤,TOP,2/8/10,"선혈포식자, 처형인의 대검, 쇠사슬 조끼, 망가진 초시계, 판금 장화, 스테락의 도전"
8,칼챔못해요,제드,MIDDLE,7/7/10,"독사의 송곳니, 충전형 물약, 드락사르의 황혼검, 명석함의 아이오니아 장화, 밤의 ..."
9,마음이 모두 닳아,카이사,BOTTOM,14/8/4,"무한의 대검, 루난의 허리케인, 징수의 총, 크라켄 학살자, 광전사의 군화"


### 트롤러 잡아내기

In [21]:
userId, userAccountId, userPuuid = searchUserId('아이번이니까렝가해주시면안될까요') # 유저의 닉네임 Giyeom
match_list = searchChampMatchId(userAccountId, '아이번') # 유저의 챔피언 나르로 설정

In [24]:
match_record(match_list[0])

,Nickname,Champion,Lane,KDA,Items
0,얼룩덜룩한말,그웬,TOP,9/2/11,"존야의 모래시계, 망각의 구, 균열 생성기, 명석함의 아이오니아 장화, 내셔의 이빨..."
1,토리5,리 신,JUNGLE,11/3/12,"곡괭이, 선혈포식자, 칠흑의 양날 도끼, 헤르메스의 발걸음, 롱소드"
2,불구경 장인,비에고,MIDDLE,6/2/9,"신성한 파괴자, 곡궁, 판금 장화, 도란의 방패, 흡혈의 낫"
3,은정곤듀님,징크스,BOTTOM,7/2/10,"도란의 검, 크라켄 학살자, 루난의 허리케인, B.F. 대검, 광전사의 군화"
4,내 인생은 고,쓰레쉬,BOTTOM,0/5/7,"강철의 솔라리 펜던트, 화이트록의 갑옷, 점화석, 기동력의 장화"
5,IlllIIIIllIlII,칼리스타,BOTTOM,1/5/8,"루난의 허리케인, 불멸의 철갑궁, 도란의 검, 광전사의 군화, 단검"
6,아이번이니까렝가해주시면안될까요,아이번,JUNGLE,0/6/9,
7,비타명재,제드,MIDDLE,6/8/2,"드락사르의 황혼검, 징수의 총, 제어 와드, 명석함의 아이오니아 장화, 롱소드, 롱소드"
8,LongStrongWard,노틸러스,BOTTOM,6/6/3,"타곤 산의 방벽, 강철의 솔라리 펜던트, 충전형 물약, 가시 갑옷, 판금 장화"
9,메우메우귀여움,스웨인,TOP,1/8/3,"도란의 반지, 리안드리의 고뇌, 장화, 망각의 구, 제어 와드"


In [27]:
a = requests.get('https://kr.api.riotgames.com/lol/match/v4/matches/' + str(match_list[0]), headers=request_headers).json()

In [33]:
a['participants'][6]

{'participantId': 7,
 'teamId': 200,
 'championId': 427,
 'spell1Id': 11,
 'spell2Id': 4,
 'stats': {'participantId': 7,
  'win': False,
  'item0': 0,
  'item1': 0,
  'item2': 0,
  'item3': 0,
  'item4': 0,
  'item5': 0,
  'item6': 0,
  'kills': 0,
  'deaths': 6,
  'assists': 9,
  'largestKillingSpree': 0,
  'largestMultiKill': 0,
  'killingSprees': 0,
  'longestTimeSpentLiving': 462,
  'doubleKills': 0,
  'tripleKills': 0,
  'quadraKills': 0,
  'pentaKills': 0,
  'unrealKills': 0,
  'totalDamageDealt': 312560,
  'magicDamageDealt': 11653,
  'physicalDamageDealt': 7063,
  'trueDamageDealt': 293843,
  'largestCriticalStrike': 0,
  'totalDamageDealtToChampions': 8830,
  'magicDamageDealtToChampions': 5046,
  'physicalDamageDealtToChampions': 3129,
  'trueDamageDealtToChampions': 655,
  'totalHeal': 4179,
  'totalUnitsHealed': 4,
  'damageSelfMitigated': 8020,
  'damageDealtToObjectives': 13182,
  'damageDealtToTurrets': 0,
  'visionScore': 39,
  'timeCCingOthers': 42,
  'totalDamageTaken